## 보배드림 크롤링
- 문제1: 날짜 지정은 어떻게 해야할까?
> `등록일`을 크롤링해서 날짜를 구분하자!!
> "/"로 split했을 때 list len이 2이면 월일 구분, 1이면 오늘
- 문제2: 몇 년도인지는 어떻게 구분할까?
> 게시글 안의 날짜 크롤링!
- 문제3: 댓글의 필요없는 앞뒤 공백들을 어떻게 없앨까?
> `strip` 함수 사용!!
- 문제4: 댓글 크롤링시 댓글, 대댓글이 같이 긁어져옴
- 문제5: 댓글이 아주 많아서 댓글 페이지가 바뀌는 경우에는??(자바스크립트로 되어있음)

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

In [3]:
# 게시판 url 지정
base_url = "http://www.bobaedream.co.kr"
page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
page_url2 = "&page="

# 검색어 & page
search = "호환"
page_num = "1"

url = base_url + page_url1 + search + page_url2 + page_num
url

'http://www.bobaedream.co.kr/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key=호환&page=1'

In [195]:
# 해당 검색어에 대한 게시글의 html 객체 가져오기
req = requests.get(url).content
doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")
# doc

In [198]:
# 게시글 url 가져오기
article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
len(article_list)
# article_url = [ i.get("href") for i in article_list ]
# article_url
# len(article_url)

98

In [161]:
# 날짜 기져오기(첫 페이지에서 best글들은 제외!!)
date = [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.num01")) == 1 ]
date

['02/19',
 '02/16',
 '02/09',
 '02/03',
 '01/29',
 '01/28',
 '01/28',
 '01/23',
 '01/22',
 '01/19',
 '01/14',
 '01/11',
 '01/11',
 '01/08',
 '01/05',
 '12/31',
 '12/31',
 '12/30',
 '12/29',
 '12/17',
 '12/12',
 '12/10',
 '12/10',
 '12/09',
 '12/05',
 '11/23',
 '11/22',
 '11/22',
 '11/22',
 '11/04',
 '10/01',
 '09/27',
 '09/13',
 '09/06',
 '08/22',
 '08/17',
 '07/02',
 '06/21',
 '06/16',
 '06/04',
 '05/24',
 '04/17',
 '04/06',
 '04/02',
 '02/24',
 '02/04',
 '09/10',
 '09/01',
 '08/30',
 '07/04',
 '03/31',
 '03/17',
 '03/14',
 '02/22',
 '02/14',
 '02/13',
 '01/29',
 '01/09',
 '12/26',
 '12/17',
 '12/14',
 '12/10',
 '11/26',
 '11/18',
 '11/13',
 '10/04',
 '09/09',
 '08/21',
 '07/12',
 '06/28',
 '06/18',
 '06/14',
 '05/18',
 '04/05',
 '01/25',
 '12/04',
 '12/02',
 '09/11',
 '07/15',
 '05/17',
 '04/27',
 '03/31',
 '03/30',
 '02/20',
 '01/13',
 '11/24',
 '10/27',
 '10/27',
 '10/07',
 '09/11',
 '06/28',
 '06/24',
 '05/20',
 '04/01',
 '02/18',
 '01/10',
 '01/09',
 '12/09',
 '10/24',
 '10/19']

In [7]:
# 게시글 제목, 내용, 댓글 가져오기
url_article = base_url + "/view?code=freeb&No=1554408&bm=1"
req = requests.get(url_article).content
doc = BeautifulSoup(req, "html.parser")

In [45]:
subject = "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] )
content = doc.select("#print_area > div.content02 > div")[0].text
comment_list = doc.select("li > dl > dd")
comment = [ i.text.strip() for i in comment_list ]   # 댓글에 있는 앞뒤 공백 제거
yymmdd = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]

In [47]:
subject, content, comment[0], yymmdd

('나쁜 사람들이 정말 많네요[12]',
 '\n지겨운 월요일 잘 보내고 계신가요???저는 컴퓨터 잘 모릅니다.컴퓨터가 안켜져서 출장수리를 불렀는데 메인보드가 나가서 갈아야 한다네요출장 불렀더니 12만원 달랍니다.출장비 3만원을 쳐주더라도 9만원이 메인보드 값이라는 건데 중고에, 기존제품 돌려주지도 않고...ㅎ...그와중에 알고보니\xa0메인보드가 기존에 쓰던거보다 하위 제품이고, 호환이 안된다고 cpu까지 하위 제품으로\xa0바꿔놓고 1도 언급은 없고결국은 동생이 나서서 6만원 돌려받고, 메인보드, cpu 다 돌려받았습니다만좋게 얘기해서는 말이 안통하네요..거기서 바꿔준 중고\xa0메인보드+ 중고 cpu 해도 3만원이면 될텐데..우리 메인보드 + cpu 돌려줄 생각도 없었으면서 돈도\xa012만원 까지 받아 챙길려고 하다니양심적으로 삽시다..아무리 먹고 살기 위해서 하는 장사라지만 사람이 양심은 있어야죠...?\n',
 '차카게 삽시다 ~ ㅎ',
 '2019.01.14')

In [50]:
# for문으로 검색어에 대한 게시글 url 가져오기
article_url = []
date = []
for page in range(1,10,1):
    # 게시판 url 지정
    base_url = "http://www.bobaedream.co.kr"
    page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
    page_url2 = "&page="

    # 검색어 & page
    search = "호환"
    page_num = str(page)

    url = base_url + page_url1 + search + page_url2 + page_num
    
    # 해당 검색어에 대한 게시글의 html 객체 가져오기
    req = requests.get(url).content
    doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")
    
    # 게시글 url 가져오기
    article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
    if len(article_list) == 0:
        break
    article_url = article_url + [ i.get("href") for i in article_list ]

    # 날짜 기져오기(best글, 관리자에 의해 삭제된 글은 제외!!)
    date = date + [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.pl14 > a.bsubject")) == 1 ]
    
    print(str(page) + "페이지 완료")

1페이지 완료
2페이지 완료
3페이지 완료


In [51]:
len(article_url), len(date)

(212, 212)

In [59]:
# for문으로 전체 url에 대한 글 크롤링
subject = []
content = []
comment = []
yymmdd = []
for sub_url in article_url:
    # 게시글 제목, 내용, 댓글 가져오기
    url_article = base_url + sub_url
    req = requests.get(url_article).content
    doc = BeautifulSoup(req, "html.parser")

    date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
    if int(date[:4]) <= 2016:
        print("{}개 데이터 크롤링 완료!".format( len(yymmdd) ))
        break
    else:
        yymmdd.append(date)
        subject.append( "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] ) )
        content.append( doc.select("#print_area > div.content02 > div")[0].text )
        comment_list = doc.select("li > dl > dd")
        comment.append( [ i.text.strip() for i in comment_list ] )   # 댓글에 있는 앞뒤 공백 제거

56개 데이터 크롤링 완료!


In [60]:
len(subject), len(content), len(comment), len(yymmdd)

(56, 56, 56, 56)

In [146]:
# 보배드림에서 검색어에 대한 게시글 url을 가져오는 함수 => date는 가져올 필요 없을듯??
def get_url_all(search):
    # for문으로 검색어에 대한 게시글 url 가져오기
    article_url = []
    date = []
    for page in range(1, 1000, 1):
        # 게시판 url 지정
        base_url = "http://www.bobaedream.co.kr"
        page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
        page_url2 = "&page="

        # 검색어 & page
        search = search
        page_num = str(page)

        url = base_url + page_url1 + search + page_url2 + page_num

        # 해당 검색어에 대한 게시글의 html 객체 가져오기
        req = requests.get(url).content
        doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")

        # 게시글 url 가져오기
        article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
        if len(article_list) == 0:
            print("작업 완료")
            break
        article_url = article_url + [ i.get("href") for i in article_list ]

        # 날짜 기져오기(best글, 관리자에 의해 삭제된 글은 제외!!)
        date = date + [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.pl14 > a.bsubject")) == 1 ]

        print(str(page) + "페이지 완료")
        
    return article_url, date


# 각 게시글의 제목, 내용, 댓글을 크롤링하는 함수
def get_crawling_data(article_url):
    # for문으로 전체 url에 대한 글 크롤링
    subject = []
    content = []
    comment = []
    yymmdd = []
    for ind, sub_url in enumerate(article_url):
        # 게시글 제목, 내용, 댓글 가져오기
        url_article = base_url + sub_url
        req = requests.get(url_article).content
        doc = BeautifulSoup(req, "html.parser")
        
        date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
        if int(date[:4]) <= 2016:
            break
        else:
            yymmdd.append(date)
            subject_text = doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")
            if len(subject_text) == 1:
                subject.append(subject_text[0])
            else:
                subject.append( "".join(subject_text[:-1] ) )
#             subject.append( "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] ) )
#             subject.append( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text )
            content.append( doc.select("#print_area > div.content02 > div")[0].text )
            comment_list = doc.select("li > dl > dd")
            comment.append( ",".join( [ i.text.strip() for i in comment_list ]) )   # 댓글에 있는 앞뒤 공백 제거    
        
        # 게시글 10개마다 개수 확인
        if (ind + 1) % 10 == 0:
            print("{}개 url 완료".format(ind + 1))
            
    print("{}개 데이터 크롤링 완료!".format(len(yymmdd)))
    
    return subject, content, comment, yymmdd

In [86]:
# 검색어에 대한 게시글 url 가져오기
article_url, date = get_url_all("호환")
len(article_url), len(date)

1페이지 완료
2페이지 완료
3페이지 완료
작업 완료


(212, 212)

In [147]:
# 게시글에 대한 제목, 내용, 댓글 크롤링
subject, content, comment, yymmdd = get_crawling_data(article_url)
len(subject), len(content), len(comment)

10개 url 완료
20개 url 완료
30개 url 완료
40개 url 완료
50개 url 완료
56개 데이터 크롤링 완료!


(56, 56, 56)

In [122]:
"".join( subject[0].split("[")[:-1] )

'차량 트렁크'

In [148]:
subject

['차량 트렁크',
 '무책임한 아이코스 회사',
 '질문} 샤시 롤러 규격 질문 드립니다.',
 '형님 동생분들 요즘  컴퓨터 어떻게들 사시는지요?!',
 'Nf쏘나타 lpg차량 obd 호환',
 '인터넷거래로 정품시켰는데 비슷하게생긴 싸구려가왔네요.',
 '블랙박스 sd카드  질문 좀 드립니다.',
 '조립 PC 사려는데 도움줄수 있는 분 계신가요!',
 '컴조립 고수님 사양 좀 봐주세요',
 '컴잘알 형님들 컴퓨터 견적 질문있어요..ㅠ',
 '나쁜 사람들이 정말 많네요',
 '네이게이션 SD 카드 문의좀 드리겠습니다.',
 '※※ ㅎㅂ주의!!!!!!!!!!!!! 어디서 타는 냄새 나지 않니?',
 '구형 소렌토 소모품 나눔해도 될까요',
 'Usb로tv연결해서보는거 도움부탁드립니다',
 'yf소나타 오디오 본체 구형 k5 오디오 패널 호환될까요?',
 '아 배고파',
 'YF소나타 사제 내비 매립',
 '혹시 뭐가 문제 인지 아시는분... 도움을 구합니다..',
 '타이어 어디까지 호환되는지 알려주실 수 있나요?',
 'R172 SLC200 타이어 어디까지 호환이 되나요?',
 '집에...에어컨팔려고하는데...',
 '보배회원님들 테레비문제로 궁금핟게있습니다',
 '블랙핑크는 투애니원 역대 노래 100% 호환 가능하지 않을까요',
 '벨로스터구형수동 미션과 i30 2.0 수동미션 파워트레인호환성',
 '점화 플러그 호환되는게 있나요??',
 '윈터타이어 사이즈 호환문의',
 '컴퓨터 그래픽카드/메인보드 호환',
 '단말기 호환 문의',
 'A3에 싸구려 후방카메라 호환되나요?',
 '휠 호환 문의입니다. 알려주세요!(자동차검사관련)',
 '베라크루즈 디스크와  sm7호환',
 '블랙박스 호환 되나요??',
 '스포티지r 2륜디젤과 yf쏘나타 차량의 부품 호환여부',
 '선배님들 블랙박스도 메모리SD도 호환타나요 ?',
 '뉴오피러스와 오피러스 프리미엄 쇼바호환되나요?',
 'tg 2.7 가솔린 lpi 점화플러그 호환되나요??',
 '블랙박스 호환? 잘아시

In [149]:
import pandas as pd

crawler_data = pd.DataFrame( {"date": yymmdd,
                              "subject": subject,
                              "content": content,
                              "comment": comment} )
crawler_data.head()

,date,subject,content,comment
0,2019.02.19,차량 트렁크,\r\n제 차량이 2007년 쎄라토 차량인데 트렁크가 찌그러져서 갈아 끼울려고 하는...,"될거같은데 도전!,안될것같은데유 ㄷㄷ 된다고해도 보기싫을꺼에유 ㄷㄷㄷ,만약 된다고 ..."
1,2019.02.16,무책임한 아이코스 회사,\r\n안녕하세요\r\n너무 화가나는 일이 있어 이렇게 글을 올려요\r\n제가 2달...,"도난 한건 본인 책임이고, 재고재품 분명있고 아이코스가 리퍼개념이라 분명 리퍼제품도..."
2,2019.02.09,질문} 샤시 롤러 규격 질문 드립니다.,\n뻘질문 죄송합니다.샤시가 오래되어 뜯어보니 롤러 수명이 다된거 같습니다.(내부 ...,"엘지 하이샤시 대리점으로 문의하세요.\n\n시중에 물건 없습니다.,답글. 감사합니다..."
3,2019.02.03,형님 동생분들 요즘 컴퓨터 어떻게들 사시는지요?!,\r\n컴퓨터 새로 살려고 하는데\r\n조립컴퓨터 조합하려고해도 각 부품호환등 이런...,"그냥 인터넷 다나와,아 들어본적있어요 검색해봐야겠네요,돈주고 삽니다 ㅋㅋㅋ 아는 동..."
4,2019.01.29,Nf쏘나타 lpg차량 obd 호환,\r\n갑자기 차량 냉각수온 계기판이 H 끝까지 올라갑니다.\r\n써모스텟이랑 수온...,"곤치는 방법 알려드릴까요?,싸구려 obd도 냉각수 온도는 나옵니다,계기판 얼마안하는..."


In [153]:
crawler_data

,date,subject,content,comment
0,2019.02.19,차량 트렁크,\r\n제 차량이 2007년 쎄라토 차량인데 트렁크가 찌그러져서 갈아 끼울려고 하는...,"될거같은데 도전!,안될것같은데유 ㄷㄷ 된다고해도 보기싫을꺼에유 ㄷㄷㄷ,만약 된다고 ..."
1,2019.02.16,무책임한 아이코스 회사,\r\n안녕하세요\r\n너무 화가나는 일이 있어 이렇게 글을 올려요\r\n제가 2달...,"도난 한건 본인 책임이고, 재고재품 분명있고 아이코스가 리퍼개념이라 분명 리퍼제품도..."
2,2019.02.09,질문} 샤시 롤러 규격 질문 드립니다.,\n뻘질문 죄송합니다.샤시가 오래되어 뜯어보니 롤러 수명이 다된거 같습니다.(내부 ...,"엘지 하이샤시 대리점으로 문의하세요.\n\n시중에 물건 없습니다.,답글. 감사합니다..."
3,2019.02.03,형님 동생분들 요즘 컴퓨터 어떻게들 사시는지요?!,\r\n컴퓨터 새로 살려고 하는데\r\n조립컴퓨터 조합하려고해도 각 부품호환등 이런...,"그냥 인터넷 다나와,아 들어본적있어요 검색해봐야겠네요,돈주고 삽니다 ㅋㅋㅋ 아는 동..."
4,2019.01.29,Nf쏘나타 lpg차량 obd 호환,\r\n갑자기 차량 냉각수온 계기판이 H 끝까지 올라갑니다.\r\n써모스텟이랑 수온...,"곤치는 방법 알려드릴까요?,싸구려 obd도 냉각수 온도는 나옵니다,계기판 얼마안하는..."
5,2019.01.28,인터넷거래로 정품시켰는데 비슷하게생긴 싸구려가왔네요.,\n정수기필터 정품 9만원넘는걸시켯는데 정품과비슷하게생긴 4만원대호환되는거싸구려가왔...,"아 업자가 물건받고 보통 환불해즈드라고요\n좀만 기다려보심이,저번주목요일날 아침에 ..."
6,2019.01.28,블랙박스 sd카드 질문 좀 드립니다.,\r\n안녕들하세요. 질문 그대로 2017년5월에 아이나비v700 16g 로...,"전용 메모리는 개소리입니다. 일반 메모리 사서 끼우세요.,고맙습니다.^^,24시간 ..."
7,2019.01.23,조립 PC 사려는데 도움줄수 있는 분 계신가요!,\n가성비 고려해서 브랜드 PC보다 조립 PC 처음 사보려고 하는데요~가격은 모니터...,타사이트 언급해서 좀그런데 뽐x 컴퓨터포럼 가시면 공지사항에 거기 회원분이 용도에따...
8,2019.01.22,컴조립 고수님 사양 좀 봐주세요,\n이번에 이사하면서 컴하나 맞추려고 합니다.여긴 보배이니 컴고수님들이 많아 사양 ...,메인보드 360이면 충분\n\n메모리 16g 짜리 한장으로\n\n최소 1테라 HDD...
9,2019.01.19,컴잘알 형님들 컴퓨터 견적 질문있어요..ㅠ,\n혼자 유튭 보면서 끙끙대면서 했어요호환이나.. 문제 될것 있을까요..??조공이 ...,용도가 어디 사용하실것인지두 적어 두시면 좋아유~\n\n게임용처럼 보이기는 하네유`...


In [152]:
crawler_data.isna().sum()   # 결측치 없음!!(but comment의 경우는 값이 없는 경우가 있으나 빈칸으로 들어가있음)

date       0
subject    0
content    0
comment    0
dtype: int64

In [5]:
driver.get(url)

In [4]:
# 웹드라이브로 크롬브라우즈 띄운다.
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
driver.implicitly_wait(3)         # 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다

In [1]:
base_url = "http://kin.naver.com/search/list.nhn?&query="
per_url_1 = "&period="
per_url_2 = "%7C"
page_url = "&page="

query = "셧다운"
per1 = "2018.01.01."
per2 = "2018.01.23."
page = str(1)
driver.get(base_url + query + per_url_1 + per1 + per_url_2 + per2 + page_url + page)      # 네이버 지식인 1page로 이동

http://kin.naver.com/search/list.nhn?&query=셧다운&period=2018.01.01.%7C2018.01.23.&page=1


In [76]:
# 1 페이지의 url 가져오기
article_list = driver.find_elements_by_css_selector('dl > dt > a')
article_urls = [ i.get_attribute('href') for i in article_list ]
article_urls

['http://kin.naver.com/qna/detail.nhn?d1id=9&dirId=9020201&docId=292966571&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=1&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=4&dirId=409&docId=292962524&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=2&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=61302&docId=292965291&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=3&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=614&docId=292972107&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=4&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40102&docId=293017325&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=5&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=814&docId=292719046&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=6&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=9&dirId=9020201&docId=293007303&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=7&search_sort=0&spq=1',
 'http://kin.naver.com/qna/de